In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# import library

In [ ]:
! pip install wxee rioxarray rasterio

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import xarray as xr
import ee
import wxee
import rioxarray
import rasterio
import pickle
import pyproj

# sample data

In [ ]:
soil_wc = xr.open_dataset('/content/drive/MyDrive/w/soil_water_content_1979001.nc')

# set coordinate

In [ ]:
LAT_MIN = 25
LAT_MAX = 45
LON_MIN = -125
LON_MAX = -65

# Authorize in Google Earth Engine

In [ ]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file='/content/drive/MyDrive/w/private-key.json'
)

ee.Initialize(credentials)

# function for mask netcdf files

In [ ]:
def mask_nc_file(
    nc,
    variable,
    lat_min,
    lat_max,
    lon_min,
    lon_max
):
    with xr.open_dataset(nc) as xr_nc:
        result = xr_nc.sel(
            lat = slice(lat_max, lat_min),
            lon = slice(lon_min, lon_max),
        )
    return result

# Load Data Curve Number

In [ ]:
crs = 4326
shape = ee.Geometry.Rectangle([LON_MAX,LAT_MIN,LON_MIN,LAT_MAX])

CN_Wyoming =  ee.Image("users/jaafarhadi/GCN250/GCN250Dry").select('b1').rename('dry')

CN_Wyoming = CN_Wyoming.clip(shape)


CN_Wyoming = CN_Wyoming.set("system:time_start", ee.Date("1979-01-01"))
arr = CN_Wyoming.wx.to_xarray(scale=10000, crs=f'EPSG:{crs}')
xds_repr_match_cn = arr.rio.reproject_match(soil_wc, resampling = rasterio.enums.Resampling.nearest)


# save as a netcdf file

In [ ]:
xds_repr_match_cn.to_netcdf("/content/drive/MyDrive/w/curve_number_1979001.nc")